### Ensemble-based models to predict age

- Random forest
- XGBoost

In [ ]:
#load in modules
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
# open latent spaces saved
latent_spaces = np.load('gtex_latent_spaces.npz')
pca_result_2d = latent_spaces['pca_2d']
pca_result_85 = latent_spaces['pca_hd']
umap_result_2d = latent_spaces['umap_from_2d']
umap_result_hd = latent_spaces['umap_from_hd']

### Fit ensemble models to predict age

In [ ]:
# Random forest

In [ ]:
# XGBoost 